# Shane's Swing Trading Strategy

* Stocks have support levels
    * e.g. MSFT hits recent low of $400 on May 1st, then hits a similar low of $390 on Aug 5th
    * Stocks reaching support levels is a buying opportunity
* However, good stocks trend up
    * This means the support level of good stocks rise over time
    * Thus it's difficult to rely on historical price data to determine support levels
* To adjust for this, we calculate support levels not by historical price but rather historical difference between price and 200 SMA
    * 200 SMA is a good indicator for the general direction of a stock
    * The difference between the price and the 200 SMA then measures the residual of the price from its expected price

In [ ]:
import alpaca_trade_api as alpaca
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, LimitOrderRequest, TakeProfitRequest, StopLossRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderClass
from dotenv import load_dotenv
import os
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import time
import datetime as dt

In [ ]:
# initialize API from API keys in .env
load_dotenv()
api_key = os.environ['APCA-API-KEY-ID']
api_secret_key = os.environ['APCA-API-SECRET-KEY']
api_base_url = 'https://paper-api.alpaca.markets'
api = alpaca.REST(api_key, api_secret_key, api_base_url)
account = api.get_account()
trading_client = TradingClient(api_key, api_secret_key, paper=True)

In [ ]:
def market_buy_order(ticker, quantity):
    time.sleep(0.2)
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=quantity,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                        )
    market_order = trading_client.submit_order(order_data=market_order_data)
    print(market_order)
    
def market_sell_order(ticker, quantity):
    time.sleep(0.2)
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=quantity,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.DAY
                        )
    market_order = trading_client.submit_order(order_data=market_order_data)
    print(market_order)

def centered_bracket_sell_order(ticker, quantity, price, radius):
    time.sleep(0.2)
    limit_order = TakeProfitRequest(
        limit_price=round(price * (1 + radius), 2)
    )
    stop_loss = StopLossRequest(
        stop_price=round(price * (1 - radius), 2)
    )
    limit_order_data = LimitOrderRequest(
        symbol=ticker,
        qty=quantity,
        side=OrderSide.SELL,
        type="limit",
        time_in_force=TimeInForce.GTC,
        order_class = 'oco',
        take_profit=limit_order,
        stop_loss=stop_loss
    )
    market_order = trading_client.submit_order(order_data=limit_order_data)
    print(market_order)

In [ ]:
def get_SP500():
    return pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

def add_business_days(start_date: dt.date, n: int) -> dt.date:
    weeks, extra_days = divmod(n, 5)
    result_date = start_date + dt.timedelta(weeks=weeks)

    if extra_days != 0:
        start_weekday = result_date.weekday()
        
        if start_weekday + extra_days > 4:
            extra_days += 2
        result_date += dt.timedelta(days=extra_days)
    return result_date

def bars_last_n_days(ticker, days):
    now = dt.datetime.now(dt.timezone.utc)
    todays_date = now.strftime("%Y-%m-%d")
    start_date = (now - dt.timedelta(days=days)).strftime("%Y-%m-%d")
    fh_date_time = start_date + "T14:30:00Z"
    bars = api.get_bars(ticker, alpaca.TimeFrame.Day, fh_date_time, adjustment='split').df
    return bars

def bars_last_n_weeks(ticker, weeks):
    now = dt.datetime.now(dt.timezone.utc)
    todays_date = now.strftime("%Y-%m-%d")
    start_date = (now - dt.timedelta(weeks=weeks)).strftime("%Y-%m-%d")
    fh_date_time = start_date + "T14:30:00Z"
    bars = api.get_bars(ticker, alpaca.TimeFrame.Day, fh_date_time, adjustment='split').df
    bars = bars.reset_index()
    bars = bars.rename(columns={'timestamp': 'date'})
    bars['date'] = pd.to_datetime(bars['date']).dt.date
    bars = bars.set_index('date')
    return bars

def add_SMA(bars, days):
    bars[f"{days} day SMA"] = ta.SMA(bars['close'], timeperiod=days)
    bars[f"close - {days} day SMA"] = bars['close'] - bars[f"{days} day SMA"]
    return bars[bars[f"{days} day SMA"].notna()]

def SMA_slope(bars, sma_days):
    time_period = 5
    return (bars.iloc[-1][f"{sma_days} day SMA"] - bars.iloc[-1 - time_period][f"{sma_days} day SMA"]) / time_period

def get_troughs(bars, n_troughs, radius):
    trough_dates = []
    sorted_bars = bars.sort_values('close - 200 day SMA')
    for i in range(len(sorted_bars)):
        close_prox = False
        row = sorted_bars.iloc[i]
        date = row.name
        for tdate in trough_dates:
            lower_bound = add_business_days(tdate, -radius)
            upper_bound = add_business_days(tdate, radius)
            if date >= lower_bound and date <= upper_bound:
                close_prox = True
                break
        if close_prox:
            continue
        trough_dates.append(date)
        if len(trough_dates) >= n_troughs:
            break
    return sorted_bars[sorted_bars.index.isin(trough_dates)].sort_index()

In [ ]:
def close_finished_positions():
    sell_threshold = 0.03

    all_positions = trading_client.get_all_positions()
    for position in all_positions:
        avg_cost_basis = float(position.avg_entry_price)
        current_price = float(position.current_price)
        if current_price > avg_cost_basis * (1 + sell_threshold) or current_price < avg_cost_basis * (1 - sell_threshold):
            market_sell_order(position.qty)
            print(f"sold {position.symbol}")

def execute_long_trading_strategy():
    troughs_std_threshold = 0.025
    SMA_slope_yearly_threshold = 0.10
    buy_amount = 3000
    price_lower_bound = 5
    price_upper_bound = 1500
    profit_taking_radius = 0.03

    held_tickers = []
    all_positions = trading_client.get_all_positions()
    for position in all_positions:
        held_tickers.append(position.symbol)

    SP500 = get_SP500()
    # tickers = pd.read_csv('trading_tickers.csv')
    for ticker in SP500['Symbol']:
        time.sleep(0.05)
        bars = bars_last_n_weeks(ticker, 85)
        print(ticker)
        bars = add_SMA(bars, 200)
        if bars.empty:
            continue
        troughs = get_troughs(bars, 7, 5)
        # troughs_range = troughs['close - 200 day SMA'].max() - troughs['close - 200 day SMA'].min()
        troughs_std = troughs['close - 200 day SMA'].std()
        troughs_mean = troughs['close - 200 day SMA'].mean()
        latest_trade = api.get_latest_trade(ticker)
        current_price = latest_trade.p
        last_sma = bars.iloc[-1]['200 day SMA']
        SMA_200_slope_percentage = SMA_slope(bars, 200) / current_price
        
        if current_price < price_lower_bound:
            continue
        if current_price > price_upper_bound:
            continue
        if troughs_std / current_price > troughs_std_threshold:
            continue 
        if current_price - last_sma > troughs_mean:
            continue
        if SMA_200_slope_percentage * 365 < SMA_slope_yearly_threshold:
            continue
        if ticker in held_tickers:
            print("repeat ticker")
            continue
        shares_to_buy = round(buy_amount / current_price)
        market_buy_order(ticker, shares_to_buy)
        time.sleep(2)
        centered_bracket_sell_order(ticker, shares_to_buy, current_price, profit_taking_radius)
        print(f"bought {ticker}")

# def execute_short_trading_strategy():
#     peaks_std_threshold = 0.025
#     SMA_slope_yearly_threshold = -0.20
#     buy_amount = 3000
#     price_lower_bound = 5
#     price_upper_bound = 1500
#     profit_taking_radius = 0.03

#     held_tickers = []
#     all_positions = trading_client.get_all_positions()
#     for position in all_positions:
#         held_tickers.append(position.symbol)

#     SP500 = get_SP500()
#     # tickers = pd.read_csv('trading_tickers.csv')
#     for ticker in SP500['Symbol']:
#         time.sleep(0.05)
#         bars = bars_last_n_weeks(ticker, 85)
#         print(ticker)
#         bars = add_SMA(bars, 200)
#         if bars.empty:
#             continue
#         troughs = get_troughs(bars, 7, 5)
#         # troughs_range = troughs['close - 200 day SMA'].max() - troughs['close - 200 day SMA'].min()
#         troughs_std = troughs['close - 200 day SMA'].std()
#         troughs_mean = troughs['close - 200 day SMA'].mean()
#         latest_trade = api.get_latest_trade(ticker)
#         current_price = latest_trade.p
#         last_sma = bars.iloc[-1]['200 day SMA']
#         SMA_200_slope_percentage = SMA_slope(bars, 200) / current_price
        
#         if current_price < price_lower_bound:
#             continue
#         if current_price > price_upper_bound:
#             continue
#         if troughs_std / current_price > troughs_std_threshold:
#             continue 
#         if current_price - last_sma > troughs_mean:
#             continue
#         if SMA_200_slope_percentage * 365 < SMA_slope_yearly_threshold:
#             continue
#         if ticker in held_tickers:
#             print("repeat ticker")
#             continue
#         shares_to_buy = round(buy_amount / current_price)
#         market_buy_order(ticker, shares_to_buy)
#         centered_bracket_sell_order(ticker, shares_to_buy, current_price, profit_taking_radius)
#         print(f"bought {ticker}")
    

In [ ]:
execute_long_trading_strategy()

In [ ]:
ticker = 'META'
bars = bars_last_n_weeks(ticker, 85)
bars = add_SMA(bars, 200)
print(SMA_slope(bars, 200))
display(bars)
bars.sort_values('close - 200 day SMA').head(20)

In [ ]:
print(SMA_slope(bars, 200))
slope = SMA_slope(bars, 200)
latest_trade = api.get_latest_trade(ticker)
current_price = latest_trade.p
print(slope / current_price)
print(slope / current_price * 365)

In [ ]:
plt.plot(bars['close - 200 day SMA'])
plt.show()
plt.clf()

In [ ]:
troughs = get_troughs(bars, 7, 5)
troughs_range = troughs['close - 200 day SMA'].max() - troughs['close - 200 day SMA'].min()
troughs_std = troughs['close - 200 day SMA'].std()
troughs_mean = troughs['close - 200 day SMA'].mean()
print(troughs_range)
print(troughs_std)
print(troughs_mean)
print(bars.iloc[-1]['200 day SMA'])
troughs